# Lab 5 Rong Xu

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [40]:
df = pd.read_csv("GSS.2006.csv")
df.head(5)

,vpsu,vstrat,adults,ballot,dateintv,famgen,form,formwt,gender1,hompop,...,away7,gender14,old14,relate14,relhh14,relhhd14,relsp14,where12,where6,where7
0,1,1957,1,3,316,2,1,1,2,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1957,2,2,630,1,2,1,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1957,2,2,314,2,1,1,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1957,1,1,313,1,2,1,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,1957,3,1,322,2,2,1,2,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 1. Run a multiple linear probability model (have at least 2 Xs in the model). Tell me how you think your independent variables will affect your dependent variable. Interpret your results. Were your expectations correct? Why or why not?

### for this question, i want to know how different between married and unmarried people behave on attending church. For attend status, i recode to 1 as offen attend religious service at least every week or even more, 0 as not often. My expectation is that people who are in married status are more likely to attend religious service than someone who is not in married status and someone who consider themselves as a very religious person will also attend religious service oftenly than someone who are not think themseleve is a very religious person. 

How often do you attend religious services? (ATTEND)

0) Never	1020	22.6
1) Less than once a year	302	6.7
2) About once or twice a year	571	12.7
3) Several times a year	502	11.1
4) About once a month	308	6.8
5) 2-3 times a month	380	8.4
6) Nearly every week	240	5.3
7) Every week	839	18.6
8) Several times a week

Are you currently -- married, widowed, divorced, separated, or have you never been married? (MARITAL)

1) Married	2170	48.1
2) Widowed	366	8.1
3) Divorced	732	16.2
4) Separated	156	3.5
5) Never married

To what extent do you consider yourself a religious person? Are you . . . (RELPERSN) from 1 very religious to 4 not at all
1) Very religious	572	19.1
2) Moderately religious	1285	42.9
3) Slightly religious	669	22.4
4) Not religious at all

In [41]:
#recode attend
conditions = [
    (df['attend'] < 7),
    (df['attend'] > 6)]
choices = [0,1] # 0 means not often, 1 means offen that goes at least every week or even more
df['attend1'] = np.select(conditions, choices, default=np.nan)
df.attend1.describe()

count    4491.000000
mean        0.260076
std         0.438725
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
Name: attend1, dtype: float64

In [42]:
df.attend1.value_counts(normalize=True)

0.0    0.739924
1.0    0.260076
Name: attend1, dtype: float64

In [43]:
lm1 = smf.ols(formula = 'attend1 ~ marital + relpersn', data = df).fit()
# 1 means married and later than are different status but not in married
lm1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                attend1   R-squared:                       0.195
Model:                            OLS   Adj. R-squared:                  0.195
Method:                 Least Squares   F-statistic:                     359.6
Date:                Sun, 26 Nov 2023   Prob (F-statistic):          1.56e-140
Time:                        01:31:01   Log-Likelihood:                -1444.8
No. Observations:                2963   AIC:                             2896.
Df Residuals:                    2960   BIC:                             2914.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.7663      0.021     36.722      0.000       0.725       0.807
marital       -0.0190      0.004     -4.288      0.000      -0.028      -0.010
relpersn      -0.1963      0.008    -25.481      0.000      -0.211      -0.181
==============================================================================
Omnibus:                      348.907   Durbin-Watson:                   1.972
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              309.367
Skew:                           0.714   Prob(JB):                     6.64e-68
Kurtosis:                       2.317   Cond. No.                         11.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### from this LPM model, I am trying to see how marriage status and personal evaluation on religious will affect their actual behavior of attend religious service. R-square = 0.195, it shown that about 19.5% of the variance in attend1 is explained by the independent variables 'marital' and 'relpersn'. Constant the marital, there is negative correlation between attend1 and relpersn. If people more tend to consider themsleves not being religious, they are more likely to have statistcally significant low frequency on attending religious service, reducing by 0.1963 unit, in absolute term. Constant the relpersn, there is negative correlation between attend1 and marital. If people more tend to be a non-married status, they are more likely to have statistcally significant low frequency on attending religious service, reducing by 0.019 unit, in absolute term. In conclusion, this regression model confirm my expectation that people who are married will more likley attend religious service also people who consider themselves as a very religious person will more likley attend religious service than someone who are not. 

# 2. Run a multiple (binary) logistic model. (It can be the same as the above LPM or a new model.) If it is a new model, tell me how you think your independent variables will affect your dependent variable. Interpret your results in the logit scale. Were your expectations correct? Why or why not?

In [44]:
logit1 = sm.formula.logit(formula = "attend1 ~ marital + relpersn", data = df).fit()
logit1.summary()

Optimization terminated successfully.
         Current function value: 0.457163
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                attend1   No. Observations:                 2963
Model:                          Logit   Df Residuals:                     2960
Method:                           MLE   Df Model:                            2
Date:                Sun, 26 Nov 2023   Pseudo R-squ.:                  0.2040
Time:                        01:31:05   Log-Likelihood:                -1354.6
converged:                       True   LL-Null:                       -1701.7
Covariance Type:            nonrobust   LLR p-value:                1.684e-151
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.2655      0.153     14.823      0.000       1.966       2.565
marital       -0.1430      0.031     -4.683      0.000      -0.203      -0.083
relpersn      -1.4474      0.070    -20.743      0.000      -1.584      -1.311
==============================================================================
"""

### Pseudo R-square=0.204, means about 20.4% of the variance in the attend1 is explained by the independent variable marital and relpersn. Marital coeff = -0.1430, it presents that the change in the log-odds of attend1 for one-unit change in the marital variable and has a negative relationship between marital and attend1. Additional one unit increased on marital, constant the relpersn, ppl will decrease their attend frequeny on attending religious service with 0.143 unit at statistically significant level. Relpersn coeff = -1.4474, it presents that the change in the log-odds of attend1 for one-unit change in the relpersn variable and has a negative relationship between relpersn and attend1. Additional one unit increased on relpersn, constant the marital, ppl will decrease their attend frequeny on attending religious service with 1.4474 unit at statistically significant level.  In conclusion, this regression presented a same idea as the previous one that married people will more likley attend religious service also a very religious person will more likley attend religious service than someone who are not. 

# 3. Get odds ratios from your logit model in Question 2 and interpret some of them.

In [45]:
np.exp(logit1.params)

Intercept    9.636002
marital      0.866736
relpersn     0.235187
dtype: float64

### These are the odds-ratios, so you can see that for people who tend to not being married, the odds that they will be go to religious service goes down by 13.4%, net of other factors; for people who tend to consider themselves as not a religious person, the odds that they will be go to religious service goes down by 76.5%, net of other factors. In this case, it shown that higher values of 'maritals' and 'relpersn' are associated with a decrease in the odds of 'attend1' being 1. 

# 4. Extra Credit: Get predicted probabilities from your logit model in Question 2 for some constellations of X values and interpret the results.

In [46]:
def logit2 (logit):
    odds = np.exp(logit)
    prob = odds / (1 + odds)
    return (prob);

intercept = logit1.params.Intercept
b_marital = logit1.params.marital
b_relpersn = logit1.params.relpersn

In [47]:
# representative value for all xs
logits_attend = intercept + (3 * b_marital) + (2 * b_relpersn)
logit2(logits_attend)

0.2576331147097901

### someone who is divoced and consider themselves as a moderately religious person, has 25.76% probability of attending religious service oftenly, like at leaset every week or even more. 

In [52]:
# representative value for all xs
logits_attend = intercept + (1 * b_marital) + (2 * b_relpersn)
logit2(logits_attend)

0.31598945767056746

### someone who is married and consider themselves as a moderately religious person, has 31.60% probability of attending religious service oftenly, like at leaset every week or even more. Compared with someone who is divoced, married group have higher probability to attend religious service with high frequency.